<a href="https://colab.research.google.com/github/Metachondria/SFT_BART/blob/main/SFT_BART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing

In [ ]:
!pip install -q evaluate
!pip install -q rouge_score

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [ ]:
from transformers import  AutoTokenizer, BartForConditionalGeneration,DataCollatorForSeq2Seq, get_linear_schedule_with_warmup
import torch
from torch.utils.data import DataLoader
import torch.nn as nn

from accelerate import Accelerator
import evaluate

from tqdm.auto import tqdm
from datasets import load_dataset

accelerator = Accelerator()
device = accelerator.device

In [ ]:
torch.cuda.device_count()

2

In [ ]:
data = load_dataset("billsum", split="ca_test")

README.md:   0%|          | 0.00/7.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/91.8M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

ca_test-00000-of-00001.parquet:   0%|          | 0.00/6.12M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [ ]:
data

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 1237
})

In [ ]:
data = data.train_test_split(test_size=0.1)

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1113
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 124
    })
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
def preprocessing(text):
    input_data = tokenizer(text['text'], max_length=1024, truncation=True)
    labels = tokenizer(text['summary'], max_length=128, truncation=True)
    input_data['labels'] = labels['input_ids']
    return input_data


In [ ]:
tokenized_data = data.map(preprocessing, batched=True)

Map:   0%|          | 0/1113 [00:00<?, ? examples/s]

Map:   0%|          | 0/124 [00:00<?, ? examples/s]

In [ ]:
tokenized_data = tokenized_data.remove_columns(['text', 'summary', 'title'])

In [ ]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1113
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 124
    })
})

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
batch_size = 8

train_dataloader = DataLoader(
    dataset=tokenized_data['train'],
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    collate_fn=data_collator
)

val_dataloader = DataLoader(
    dataset=tokenized_data['test'],
    batch_size=batch_size,
    collate_fn=data_collator
)

In [ ]:
next(iter(train_dataloader)).keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

# Train Loop

In [ ]:
num_epochs = 2

optimizer = torch.optim.AdamW(model.parameters(),
                              lr = 3e-5)

total_step = len(data['train']) * num_epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=500,
    num_training_steps=total_step
)

In [ ]:
model, optimizer, train_dataloader, scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, scheduler
)

In [ ]:
for epoch in range(num_epochs):
    epoch_loss = 0
    val_epoch_loss = 0
    model.train()

    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()

        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']

        output = model(input_ids=input_ids,
                       attention_mask=attention_mask,
                       labels=labels)
        loss = output.loss

        accelerator.backward(loss)
        optimizer.step()
        scheduler.step()

        gathered_loss = accelerator.gather(loss).mean().item()
        epoch_loss += gathered_loss

    mean_epoch_loss = epoch_loss / len(train_dataloader)
    print(f'{epoch} epoch: mean_loss {mean_epoch_loss}')

    model.eval()
    total_val_loss = 0.0
    total_val_samples = 0

    for batch in tqdm(val_dataloader, desc=f"Validation Epoch {epoch}"):
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        with torch.no_grad():
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

        batch_size = input_ids.size(0)
        gathered_loss = accelerator.gather(outputs.loss.detach() * batch_size)
        total_val_loss += gathered_loss.sum().item()
        total_val_samples += accelerator.gather(torch.tensor(batch_size, device=accelerator.device)).sum().item()

    mean_val_loss = total_val_loss / total_val_samples
    print(f"Epoch {epoch} | Val Loss: {mean_val_loss:.4f}")
    print("-" * 50)




  0%|          | 0/139 [00:00<?, ?it/s]

0 epoch: mean_loss 1.4557626616182944


Validation Epoch 0:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 0 | Val Loss: 1.4879
--------------------------------------------------


  0%|          | 0/139 [00:00<?, ?it/s]

1 epoch: mean_loss 1.2151446698380888


Validation Epoch 1:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 1 | Val Loss: 1.4518
--------------------------------------------------


# Count ROUGE

In [ ]:
summaries = []

for text in tqdm(data["test"]['text']):
    input_ids = tokenizer.encode(
        text,
        return_tensors="pt",
        max_length=1024,
        truncation=True,
        ).to(device)

    summary_text_ids = model.generate(
        input_ids=input_ids,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        max_length=142,
        min_length=56,
        num_beams=4,
    )

    decoded_text = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)
    summaries.append(decoded_text)

  0%|          | 0/124 [00:00<?, ?it/s]

In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
results = rouge.compute(
        predictions=summaries,
        references=data["test"]['summary']
    )

In [ ]:
results

{'rouge1': 0.3746832315584462,
 'rouge2': 0.2243917714704478,
 'rougeL': 0.2596515446083695,
 'rougeLsum': 0.32759249645737015}

In [1]:
#Model trained for total 4 epochs (2 consecutive training sessions)